## Mini proyecto : ETL de la POKEAPI

### Extracción

Importo las librerias que voy a usar por el momento y defino una función para conectarme a la API y traerme los datos

In [1]:
import requests
import pandas as pd
from datetime import datetime

def extract_pokemon_data(limit=100, offset=0): #funcion para traerme la lista de los bichos
    url = f"https://pokeapi.co/api/v2/pokemon?limit={limit}&offset={offset}"
    try:
        response = requests.get(url)
        response.raise_for_status()  
        data = response.json()
        return data["results"], data["count"] 
    except requests.exceptions.RequestException as e:
        print(f"Error al conectar con la API: {e}")
        return [], 0

def get_pokemon_details(url): #funcion para traerme los detalles por bicho
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener detalles: {e}")
        return None

#Check
pokemon_list, total_count = extract_pokemon_data(limit=50)
print(f"Disponibles: {total_count}")
print(f"Obtuve {len(pokemon_list)} bichos")

Disponibles: 1302
Obtuve 50 bichos


### Transformación

Defino una funcion para traerme los detalles que quiero para cada bicho

In [2]:
def transform_pokemon_data(pokemon_list):
    transformed_data = []
    for pokemon in pokemon_list:
        details = get_pokemon_details(pokemon["url"])
        if details:
            # Traigo los datos que quiero ver
            types = [t["type"]["name"] for t in details["types"]]
            abilities = [a["ability"]["name"] for a in details["abilities"]]
            stats = {s["stat"]["name"]: s["base_stat"] for s in details["stats"]}
            
            transformed_data.append({
                "id": details["id"],
                "name": details["name"],
                "types": ", ".join(types), 
                "abilities": ", ".join(abilities),
                "height": details["height"],
                "weight": details["weight"],
                "hp": stats.get("hp", 0),
                "attack": stats.get("attack", 0),
                "defense": stats.get("defense", 0),
                "special-attack": stats.get("special-attack", 0),
                "special-defense": stats.get("special-defense", 0),
                "speed": stats.get("speed", 0)
            })
    return transformed_data

# Transformar los datos extraídos
transformed_data = transform_pokemon_data(pokemon_list)
#print("Primer Pokémon transformado:", transformed_data[0] if transformed_data else "No hay datos") #Prueba

## Carga

Voy a llevar a DataFrame la info para ver como quedo y la voy a guardar en un CSV

In [3]:
# Convertir a DataFrame y guardar en CSV
df = pd.DataFrame(transformed_data)
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
csv_filename = f"Datos_bichos{timestamp}.csv"
df.to_csv(csv_filename, index=False)
print(f"Datos guardados en {csv_filename}")

Datos guardados en Datos_bichos2025-05-14_09-17-37.csv


Ahora cuento con la info de la API en un DataFrame y podría analizar dicha info, pero por el momento no.

In [4]:
df.head()

,id,name,types,abilities,height,weight,hp,attack,defense,special-attack,special-defense,speed
0,1,bulbasaur,"grass, poison","overgrow, chlorophyll",7,69,45,49,49,65,65,45
1,2,ivysaur,"grass, poison","overgrow, chlorophyll",10,130,60,62,63,80,80,60
2,3,venusaur,"grass, poison","overgrow, chlorophyll",20,1000,80,82,83,100,100,80
3,4,charmander,fire,"blaze, solar-power",6,85,39,52,43,60,50,65
4,5,charmeleon,fire,"blaze, solar-power",11,190,58,64,58,80,65,80


In [5]:
df.describe()

,id,height,weight,hp,attack,defense,special-attack,special-defense,speed
count,50.00000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.00000,50.000000
mean,25.50000,9.440000,234.140000,60.040000,64.360000,58.160000,59.040000,60.88000,63.380000
std,14.57738,5.855958,239.438119,22.230665,19.876927,19.637906,24.448163,22.17338,23.215837
min,1.00000,2.000000,8.000000,10.000000,20.000000,20.000000,20.000000,20.00000,20.000000
25%,13.25000,5.250000,71.250000,45.000000,49.250000,43.250000,40.000000,46.25000,45.000000
50%,25.50000,8.500000,127.500000,60.000000,62.500000,55.000000,57.500000,64.50000,62.500000
75%,37.75000,11.750000,300.000000,72.250000,80.000000,72.250000,80.000000,79.75000,80.000000
max,50.00000,35.000000,1000.000000,140.000000,102.000000,110.000000,110.000000,105.00000,110.000000
